# Importações

In [2]:
import re
import spacy
import pickle
import os
import pandas as pd
import csv
import glob
import csv
from spacy.training import offsets_to_biluo_tags
from spacy.training import biluo_tags_to_spans
from spacy.training import offsets_to_biluo_tags, biluo_tags_to_offsets, biluo_tags_to_spans
import itertools

# importações para o bloco de código do treinamento do modelo de NER
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.tokens import Doc
from spacy.training import Example
import warnings
import datetime as date

### Carregando Dados Rotulados Manualmente Para Treinamento do Modelo

Carrega os dados no formato .csv e cria um dataframe a partir deles.

In [3]:
dataSet = pd.read_csv("FinalDataSet.csv", usecols=['Palavras', 'Rotulo', 'Sentenca', 'Inicio', 'Fim'])

Carrega a base utilizada para rotulação manual das categorias. Utilizada para extrair as setenças a que cada token rotulado pertence.

In [4]:
with open("Dados_Utilizados_Para_RotulacaoManual.txt", "r", encoding='utf-8') as file:
    textos = file.read().splitlines()
print("Quantidade de linhas da base:", len(textos))

Quantidade de linhas da base: 14287


Código para converter os dados que estão no formato IOB para o formato do spaCy

In [5]:
sMarker = 0 #Marcador que referência a qual setença a entidade pertence, ou seja, serve pra pegar o texto da referencia dos tokens no dataset
s = textos[sMarker]
TRAIN_DATA = []
entities = []
for index, row in dataSet.iterrows():
    
    if(row['Sentenca']-1 != sMarker): # Nova sentença
        if entities:
            TRAIN_DATA.append((s, {"entities": [tuple(e) for e in entities]})) # Salva sentença anterior
        entities = [] # esvazia entidades
        sMarker = row['Sentenca']-1 # atualiza o marcador de sentença
        
        if (sMarker < len(textos)): # Limite de textos
            s = textos[sMarker]

    if(row['Rotulo'][0] == 'B'):
         entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
    if(row['Rotulo'][0] == 'I'):
        if (entities): 
            entities[-1][1] = row['Fim']
        else:
            # print(index)
            entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
    
    if index == dataSet.index[-1]: # Ultimo elemento
        if entities:
            TRAIN_DATA.append((s, {"entities": [tuple(e) for e in entities]}))
        entities = [] 


len(TRAIN_DATA)

10969

In [6]:
#TRAIN_DATA = TRAIN_DATA[0:1000]
for item in TRAIN_DATA:
    print(item)

('NOME DA CACHAÇA: Cachaça Três Coronéis', {'entities': [(25, 38, 'NOME_BEBIDA')]})
('PREÇO: R$ 35,00', {'entities': [(7, 15, 'PRECO')]})
('DESCRIÇÃO DA CACHAÇA: Cachaça Três Coronéis 700ml', {'entities': [(30, 43, 'NOME_BEBIDA'), (44, 49, 'VOLUME')]})
('Volume: 750ml', {'entities': [(8, 13, 'VOLUME')]})
('Graduação Alcoólica: 39,2%', {'entities': [(21, 26, 'GRADUACAO_ALCOOLICA')]})
('RECIPIENTE DE ARMAZENAMENTO: Tipo Barril: Carvalho', {'entities': [(42, 50, 'TIPO_MADEIRA')]})
('Produtor: Glauri Ind Com. Ltda', {'entities': [(10, 30, 'NOME_ORGANIZACAO')]})
('Local de Produção: Boituva / São Paulo', {'entities': [(19, 26, 'NOME_LOCAL'), (29, 38, 'NOME_LOCAL')]})
('NOME DA CACHAÇA: Cachaça Princesa Isabel Jaqueira Ouro', {'entities': [(25, 40, 'NOME_BEBIDA'), (41, 49, 'TIPO_MADEIRA'), (50, 54, 'CLASSIFICACAO_BEBIDA')]})
('PREÇO: R$ 85,00', {'entities': [(7, 15, 'PRECO')]})
('Lá, um certo dia no ano de 2015, uma frondosa jaqueira secular tombou durante um temporal, foi quando nosso patri

('NOME DA CACHAÇA: Cachaça Dom Cabral Ed. 1500 Prata 600ml', {'entities': [(25, 35, 'NOME_BEBIDA'), (45, 50, 'CLASSIFICACAO_BEBIDA'), (51, 56, 'VOLUME')]})
('PREÇO: R$ 48,90', {'entities': [(7, 15, 'PRECO')]})
('Volume: 600ml', {'entities': [(8, 13, 'VOLUME')]})
('NOME DA CACHAÇA: Cachaça Dom Cabral Ed. 1500 Ouro 600ml', {'entities': [(25, 35, 'NOME_BEBIDA'), (45, 49, 'CLASSIFICACAO_BEBIDA'), (50, 55, 'VOLUME')]})
('PREÇO: R$ 39,99', {'entities': [(7, 15, 'PRECO')]})
('Volume: 600ml', {'entities': [(8, 13, 'VOLUME')]})
('NOME DA CACHAÇA: Cachaça Dom Cabral Prata 700ml', {'entities': [(25, 35, 'NOME_BEBIDA'), (36, 41, 'CLASSIFICACAO_BEBIDA'), (42, 47, 'VOLUME')]})
('PREÇO: R$ 59,90', {'entities': [(7, 15, 'PRECO')]})
('Volume: 700ml', {'entities': [(8, 13, 'VOLUME')]})
('NOME DA CACHAÇA: Cachaça Vale do Cedro 700ml', {'entities': [(25, 38, 'NOME_BEBIDA'), (39, 44, 'VOLUME')]})
('PREÇO: R$ 56,00', {'entities': [(7, 15, 'PRECO')]})
('DESCRIÇÃO DA CACHAÇA: A cachaça Vale do Cedro foi funda

('Cachaça Xanadu Black Ouro armazenada em tonéis de Carvalho.', {'entities': [(8, 14, 'NOME_BEBIDA'), (15, 25, 'CLASSIFICACAO_BEBIDA'), (40, 46, 'RECIPIENTE_ARMAZENAMENTO'), (50, 58, 'TIPO_MADEIRA')]})
('Tipo: Cachaça Ouro Black', {'entities': [(14, 24, 'CLASSIFICACAO_BEBIDA')]})
('Região: Blumenau/SC', {'entities': [(8, 16, 'NOME_LOCAL'), (17, 19, 'NOME_LOCAL')]})
('Produtor: Xanadu Agroindústria de Bebidas', {'entities': [(10, 41, 'NOME_ORGANIZACAO')]})
('País: Brasil', {'entities': [(6, 12, 'NOME_LOCAL')]})
('Graduação Alcóolica: 38%', {'entities': [(21, 24, 'GRADUACAO_ALCOOLICA')]})
('Volume: 750ml', {'entities': [(8, 13, 'VOLUME')]})
('Tipo: Cachaça Ouro Black', {'entities': [(14, 24, 'CLASSIFICACAO_BEBIDA')]})
('Região: Blumenau/SC', {'entities': [(8, 16, 'NOME_LOCAL'), (17, 19, 'NOME_LOCAL')]})
('País: Brasil', {'entities': [(6, 12, 'NOME_LOCAL')]})
('Volume: 700ML', {'entities': [(8, 13, 'VOLUME')]})
('Graduação Alcóolica: 38%', {'entities': [(21, 24, 'GRADUACAO_ALCOOLICA')]})


('Grad. Alcool: 44%', {'entities': [(14, 17, 'GRADUACAO_ALCOOLICA')]})
('NOME DA CACHAÇA: Cachaça Hanavilhana Copa especial 1000 ml', {'entities': [(25, 36, 'NOME_BEBIDA'), (51, 58, 'VOLUME')]})
('PREÇO: R$ 59,90', {'entities': [(7, 15, 'PRECO')]})
('Volume: 1000 ml', {'entities': [(8, 15, 'VOLUME')]})
('DESCRIÇÃO DA CACHAÇA: Cachaça Hanavilhana Copa especial 1000 mlProduzida na cidade de Salinas (MG)Armazenado em tonéis de UmburanaConteúdo : 1000 mlGarrafa : Especial copa do mundo transparentePrazo de Validade : IndeterminadoGraduação : 42 %Evite o consumo Excessivo de Álcool', {'entities': [(30, 41, 'NOME_BEBIDA'), (56, 60, 'VOLUME'), (86, 93, 'NOME_LOCAL'), (95, 97, 'NOME_LOCAL'), (112, 118, 'RECIPIENTE_ARMAZENAMENTO'), (141, 145, 'VOLUME'), (238, 240, 'GRADUACAO_ALCOOLICA')]})
('Grad. Alcool: 42%', {'entities': [(14, 17, 'GRADUACAO_ALCOOLICA')]})
('NOME DA CACHAÇA: Cachaça Cachoeira Copa especial 1000 ml', {'entities': [(25, 34, 'NOME_BEBIDA'), (49, 56, 'VOLUME')]})
('PREÇO: R$ 59,

('DESCRIÇÃO DA CACHAÇA: Canelinha Kenga Dom Tápparo 1 Litro Origem: Mirassol/SP Grad.', {'entities': [(22, 37, 'NOME_BEBIDA'), (38, 49, 'NOME_BEBIDA'), (50, 57, 'VOLUME'), (66, 74, 'NOME_LOCAL'), (75, 77, 'NOME_LOCAL')]})
('Alcoólica: 28,5% Volume: 1000 ml Informações do Produto: Coquetel de Canela com cachaça.', {'entities': [(11, 16, 'GRADUACAO_ALCOOLICA'), (25, 32, 'VOLUME')]})
('LOCAL DE PRODUÇÃO: Mirassol/SP', {'entities': [(19, 27, 'NOME_LOCAL'), (28, 30, 'NOME_LOCAL')]})
('NOME DA CACHAÇA: Cachaça com Caju e Hortelã Cajulã 670 ML', {'entities': [(44, 50, 'NOME_BEBIDA'), (51, 57, 'VOLUME')]})
('PREÇO: R$49,90', {'entities': [(7, 14, 'PRECO')]})
('DESCRIÇÃO DA CACHAÇA: Cachaça com Caju e Hortelã Cajulã 670 ML Bebida Mista com Fruta Origem: Senador José Bento / Minas Gerais Teor Alcoólico: 15 % Volume: 700 Ml Uma bebida extremamente refrescante, de um sabor ímpar, 100% artesanal, produzido com cachaça de alambique.', {'entities': [(49, 55, 'NOME_BEBIDA'), (56, 62, 'VOLUME'), (94, 1

...

('NOME DA CACHAÇA: Cachaça Santo Grau Solera Cinco Botas', {'entities': [(25, 54, 'NOME_BEBIDA')]})
('PREÇO: R$ 149,90', {'entities': [(7, 16, 'PRECO')]})
('DESCRIÇÃO DA CACHAÇA: Cachaça Santo Grau Solera Cinco Botas 750 ml.', {'entities': [(30, 59, 'NOME_BEBIDA'), (60, 66, 'VOLUME')]})
('Da linha Cachaças Raras de Engenho, a Santo Grau Solera Cinco Botas é uma cachaça de alambique envelhecida por meio do tradicional sistema único de Solera.', {'entities': [(18, 34, 'NOME_BEBIDA'), (38, 67, 'NOME_BEBIDA'), (85, 94, 'EQUIPAMENTO_DESTILACAO')]})
('Envelhecida nos barris (ou “botas”, em espanhol) que por anos armazenaram o tradicional vinho de Jerez Oloroso, da bicentenária empresa espanhola Osborne.', {'entities': [(16, 22, 'RECIPIENTE_ARMAZENAMENTO'), (146, 153, 'NOME_ORGANIZACAO')]})
('Volume: 750 ml', {'entities': [(8, 14, 'VOLUME')]})
('GRADUAÇÃO ALCOÓLICA: 42% vol.', {'entities': [(21, 24, 'GRADUACAO_ALCOOLICA')]})
('NOME DA CACHAÇA: Cachaça Santo Grau Itirapuã', {'entities': [(25, 

('Coloração: Amarelo Dourado (para caramelo) – ótimo corpo – límpida e brilhante', {'entities': [(11, 26, 'CARACTERISTICA_SENSORIAL_COR'), (33, 41, 'CARACTERISTICA_SENSORIAL_COR'), (45, 56, 'CARACTERISTICA_SENSORIAL_CONSISTÊNCIA'), (59, 66, 'CARACTERISTICA_SENSORIAL_COR'), (69, 78, 'CARACTERISTICA_SENSORIAL_COR')]})
('Aroma: adocicados – frutado (banana baunilha – finaliza com menta no início) – amadeirados – coco', {'entities': [(7, 17, 'CARACTERISTICA_SENSORIAL_AROMA'), (20, 27, 'CARACTERISTICA_SENSORIAL_AROMA'), (29, 35, 'CARACTERISTICA_SENSORIAL_AROMA'), (36, 44, 'CARACTERISTICA_SENSORIAL_AROMA'), (60, 65, 'CARACTERISTICA_SENSORIAL_AROMA'), (79, 90, 'CARACTERISTICA_SENSORIAL_AROMA'), (93, 97, 'CARACTERISTICA_SENSORIAL_AROMA')]})
('Gostos: Doce', {'entities': [(8, 12, 'CARACTERISTICA_SENSORIAL_SABOR')]})
('Sensações: aveludado – levemente picante', {'entities': [(11, 20, 'CARACTERISTICA_SENSORIAL_CONSISTÊNCIA'), (23, 40, 'CARACTERISTICA_SENSORIAL_SABOR')]})
('Retrogosto: agradável (

('Cachaça Tiê Ouro é produzida em Aiuruoca, cidade localizada no sul de Minas Gerais,', {'entities': [(8, 11, 'NOME_BEBIDA'), (12, 16, 'CLASSIFICACAO_BEBIDA'), (32, 40, 'NOME_LOCAL'), (63, 82, 'NOME_LOCAL')]})
('Minas Gerais - Brasil', {'entities': [(0, 12, 'NOME_LOCAL'), (15, 21, 'NOME_LOCAL')]})
('Após a destilação, a Cachaça Tiê Ouro é envelhecida em barris de carvalho europeu de 200 litros por 1 ano e meio.', {'entities': [(29, 32, 'NOME_BEBIDA'), (33, 37, 'CLASSIFICACAO_BEBIDA'), (55, 61, 'RECIPIENTE_ARMAZENAMENTO'), (65, 81, 'TIPO_MADEIRA'), (85, 95, 'VOLUME'), (100, 112, 'TEMPO_ARMAZENAMENTO')]})
('Produção sustentável de cachaça  Tiê Ouro, no sul de Minas Gerais', {'entities': [(33, 36, 'NOME_BEBIDA'), (37, 41, 'CLASSIFICACAO_BEBIDA'), (46, 65, 'NOME_LOCAL')]})
('A Cachaça Tiê Ouro é produzida em Aiuruoca, cidade localizada no sul de Minas Gerais, em uma bonita região de importância ambiental e histórica.', {'entities': [(10, 13, 'NOME_BEBIDA'), (14, 18, 'CLASSIFICACAO_BEBIDA')

('PREÇO: R$ 53,90', {'entities': [(7, 15, 'PRECO')]})
('VOLUME: 1000ml', {'entities': [(8, 14, 'VOLUME')]})
('DESCRIÇÃO DA CACHAÇA: 1000ml', {'entities': [(22, 28, 'VOLUME')]})
('umburana', {'entities': [(0, 8, 'TIPO_MADEIRA')]})
('Graduação alcoólica: 39% Bidestilada', {'entities': [(21, 24, 'GRADUACAO_ALCOOLICA')]})
('Cor: Amarelo-palha', {'entities': [(5, 18, 'CARACTERISTICA_SENSORIAL_COR')]})
('De 6 meses a 01 ano.', {'entities': [(3, 10, 'TEMPO_ARMAZENAMENTO'), (13, 19, 'TEMPO_ARMAZENAMENTO')]})
('Madeira: Umburana', {'entities': [(9, 17, 'TIPO_MADEIRA')]})
('Visual: Amarelo palha brilhante.', {'entities': [(8, 21, 'CARACTERISTICA_SENSORIAL_COR'), (22, 31, 'CARACTERISTICA_SENSORIAL_COR')]})
('Olfativa: Muito fina, intensa e persistente. Nariz delicado, com boa fruta, notas típicas de canela e leves de amêndoas.', {'entities': [(69, 74, 'CARACTERISTICA_SENSORIAL_AROMA'), (93, 99, 'CARACTERISTICA_SENSORIAL_AROMA'), (111, 119, 'CARACTERISTICA_SENSORIAL_AROMA')]})
('Gustativa: Muito m

('Sua elaboração é totalmente artesanal, dotada de segredos e técnicas de gente que vive em meio à produção de cachaça desde que nasceu, conservando a essência das cachaças de Salinas.Madeira: InoxGrad.', {'entities': [(174, 181, 'NOME_LOCAL')]})
('Alcoólica: 40%\u200bVolume: 750 mlTempo de Envelhecimento: n/aOrigem: Salinas/MGColoração: Branca incolorAnálise: Aroma que remete a cana.', {'entities': [(23, 26, 'VOLUME'), (65, 72, 'NOME_LOCAL'), (86, 92, 'CARACTERISTICA_SENSORIAL_COR'), (128, 132, 'CARACTERISTICA_SENSORIAL_AROMA')]})
('Volume: 750ml', {'entities': [(8, 13, 'VOLUME')]})
('NOME DA CACHAÇA: Cachaça Cacharioca tradicional', {'entities': [(25, 47, 'NOME_BEBIDA')]})
('PREÇO: R$ 39,99', {'entities': [(7, 15, 'VOLUME')]})
('DESCRIÇÃO DA CACHAÇA: A Cachaça Cacharioca é produzida na cidade mineira de Salinas, região internacionalmente reconhecida por abrigar as melhores cachaças existentes no mercado mundial.', {'entities': [(32, 42, 'NOME_BEBIDA'), (76, 83, 'NOME_LOCAL')]})
('Sua

### Configurando informações para o treinamento do modelo

In [47]:
# Carregando o modelo de nlp em português do spaCy. Esse modelo já foi baixado anteriormente

nlp = spacy.load('pt_core_news_sm')
print ("Modelo carregado '% s'"% nlp)

Modelo carregado '<spacy.lang.pt.Portuguese object at 0x00000269BB5085E0>'


In [48]:
# Obtendo o componente do pipeline para trabalhar com reconhecimentos de entidades nomeadas
ner = nlp.get_pipe('ner')

# Adicionando as categorias de entidade nomeada ao pipeline 'ner'
ner.add_label('CARACTERISTICA_SENSORIAL_AROMA')
ner.add_label('CARACTERISTICA_SENSORIAL_CONSISTÊNCIA')
ner.add_label('CARACTERISTICA_SENSORIAL_SABOR')
ner.add_label('CARACTERISTICA_SENSORIAL_COR')
ner.add_label('RECIPIENTE_ARMAZENAMENTO')
ner.add_label('EQUIPAMENTO_DESTILACAO')
ner.add_label('CLASSIFICACAO_BEBIDA')
ner.add_label('TEMPO_ARMAZENAMENTO')
ner.add_label('GRADUACAO_ALCOOLICA')
ner.add_label('TIPO_MADEIRA')
ner.add_label('NOME_BEBIDA')
ner.add_label('VOLUME')
ner.add_label('NOME_LOCAL')
ner.add_label('NOME_ORGANIZACAO')
ner.add_label('NOME_PESSOA')
ner.add_label('PRECO')
ner.add_label('TEMPO')

1

In [49]:
#verificando as categorias de entidades nomeadas contidas no pipeline
print("Categorias de entidade que o modelo contém:", '\n\n', ner.label_data)

Categorias de entidade que o modelo contém: 

 {0: Counter(), 1: Counter({'LOC': 137478, 'PER': 65030, 'MISC': 38659, 'ORG': 25003, 'CARACTERISTICA_SENSORIAL_AROMA': -1, 'CARACTERISTICA_SENSORIAL_CONSISTÊNCIA': -2, 'CARACTERISTICA_SENSORIAL_SABOR': -3, 'CARACTERISTICA_SENSORIAL_COR': -4, 'RECIPIENTE_ARMAZENAMENTO': -5, 'EQUIPAMENTO_DESTILACAO': -6, 'CLASSIFICACAO_BEBIDA': -7, 'TEMPO_ARMAZENAMENTO': -8, 'GRADUACAO_ALCOOLICA': -9, 'TIPO_MADEIRA': -10, 'NOME_BEBIDA': -11, 'VOLUME': -12, 'NOME_LOCAL': -13, 'NOME_ORGANIZACAO': -14, 'NOME_PESSOA': -15, 'PRECO': -16, 'TEMPO': -17}), 2: Counter({'LOC': 137478, 'PER': 65030, 'MISC': 38659, 'ORG': 25003, 'CARACTERISTICA_SENSORIAL_AROMA': -1, 'CARACTERISTICA_SENSORIAL_CONSISTÊNCIA': -2, 'CARACTERISTICA_SENSORIAL_SABOR': -3, 'CARACTERISTICA_SENSORIAL_COR': -4, 'RECIPIENTE_ARMAZENAMENTO': -5, 'EQUIPAMENTO_DESTILACAO': -6, 'CLASSIFICACAO_BEBIDA': -7, 'TEMPO_ARMAZENAMENTO': -8, 'GRADUACAO_ALCOOLICA': -9, 'TIPO_MADEIRA': -10, 'NOME_BEBIDA': -11, 'VOLU

In [50]:
# Obter os nomes dos componentes que queremos desativar durante o treinamento

#pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
#unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

### Treinando o modelo de reconhecimento de entidades nomeadas

In [51]:
# iniciar o ciclo de treinamento, apenas treinando NER

epochs = 2
optimizer = nlp.create_optimizer()
#optimizer = nlp.resume_training()
#optimizer = nlp.begin_training()

with nlp.disable_pipes(*unaffected_pipes), warnings.catch_warnings():
    warnings.filterwarnings("once", category=UserWarning, module='spacy')
    print(date.datetime.now())
    sizes = compounding(4.0, 32.0, 1.001)
    
    # agrupe os exemplos usando o minibatc do spaCy
    for epoch in range(epochs):
        examples = TRAIN_DATA
        random.shuffle(examples)
        batches = minibatch(examples, size=sizes)
        losses = {}
        
        for batch in batches:
            texts, annotations = zip(*batch)
            example = []
            # Atualize o modelo com a iteração de cada texto
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
                # Update the model
                nlp.update(example, drop=0.35, losses=losses,sgd=optimizer)
        print("Losses ({}/{})" .format(epoch + 1, epochs), '-', date.datetime.now(), losses)
            

2022-02-16 09:24:49.657573


C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "Premiações : 1º Lugar no concurso de cachaça de qu..." with entities "[(58, 71, 'NOME_ORGANIZACAO'), (72, 86, 'NOME_LOCA...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "DESCRIÇÃO DA CACHAÇA: Cachaça Artesanal produzida ..." with entities "[(53, 62, 'NOME_LOCAL'), (63, 65, 'NOME_LOCAL'), (...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities coul

KeyboardInterrupt: 

## Testando o Modelo

In [ ]:
# Exemplo using o display sentence involving original entities
spacy.displacy.render(nlp("A Bacuri é uma cachaça, tem um sabor tropical que vai aumentar o seu ânimo e envolvê-lo em uma onda de puro prazer. É armazenada em barris de jequitibá"), style="ent")

### Carregando alguns Dados de Teste

In [ ]:
dados_Teste = [('A Cachaça Serra Limpa Prata 355ml é uma bebida armazenada por seis meses em tonéis de inox. Robusta. É uma das únicas Cachaças Artesanais cem porcento orgânicas do Brasil. Contém 50% de graduação alcoolia',
  {'entities': [(76, 91, 'RECIPIENTE_ARMAZENAMENTO')]}),
  
  ('A Cachaça Minha Deusa Vale Verde Prata 1000ml é uma bebida armazenada por um ano em tonéis de grápia, o que realça o aroma de cana-de-açúcar. Leve,. De sabor suave e marcante, é perfeita para o preparo de drinks e coquetéis.',
  {'entities': [(84, 100, 'RECIPIENTE_ARMAZENAMENTO')]}),

  ('Cachaça Mineirinha 355ml é uma edição especial das Cachaças Dona Beja, de Perdizes, Minas Gerais. Armazenada por cinco anos em barris de jequitiba. Chama a atenção a embalagem da bebida, que é envasada em garrafas de louça que imitam a cana-de-açúcar.',
  {'entities': [(127, 147, 'RECIPIENTE_ARMAZENAMENTO')]}),
 
  ('Três Corações é uma bebida armazenada por 2 anos em jequitibá. Robusta e com aroma doce', 
  {'entities':[(52, 61, 'RECIPIENTE_ARMAZENAMENTO')]})          
    ]

### Avaliação do Modelo

In [ ]:
# Compara item por item o que foi classificado pelo modelo e a classifcação correta(manual)

#nlp = spacy.load(output_dir)
examples = []

for text, annots in dados_Teste:
    print(f'Manual Anotation: {annots}')
    doc = nlp(text)
    #doc = nlp.make_doc(text)
    spacy.displacy.render(nlp(text))
    
    for item in doc.ents:
        print('Model prediction: ', 'entities: ', [(item.start_char, item.end_char,item.label_)])
                
    examples.append(Example.from_dict(doc, annots)) 
    #print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

### Métricas Precision, Recall e f-measure: 

In [ ]:
# Apresenta o resultados das métricas para o modelo como um todo, e as métricas do modelo para cada categoria de entidade

print(nlp.evaluate(examples))

### Avaliação do Modelo pelas Métrica Acuracia

In [ ]:
# dictionary which will be populated with the entities and result information
entity_evaluation = {}

# helper function to udpate the entity_evaluation dictionary
def update_results(entity, metric):
    if entity not in entity_evaluation:
        entity_evaluation[entity] = {"correct": 0, "total": 0}
    
    entity_evaluation[entity][metric] += 1

# same as before, see if entities from test set match what spaCy currently predicts
for data in dados_Teste:
    sentence = data[0]
    entities = data[1]["entities"]

    for entity in entities:
        doc = nlp(sentence)
        correct_text = sentence[entity[0]:entity[1]]

        for ent in doc.ents:
            if ent.label_ == entity[2] and ent.text == correct_text:
                update_results(ent.label_, "correct")
                break

        update_results(entity[2], "total")
print("Resultado para cada categoria:",'\n', entity_evaluation)

In [ ]:
sum_total = 0
sum_correct = 0

for entity in entity_evaluation:
    total = entity_evaluation[entity]["total"]
    correct = entity_evaluation[entity]["correct"]

    sum_total += total
    sum_correct += correct
    
    print("{} | {:.2f}%".format(entity, correct / total * 100))

print()
print("Overall accuracy: {:.2f}%".format(sum_correct / sum_total * 100))

## Salvando o Modelo

In [ ]:
# Save the  model to directory

from pathlib import Path
output_dir=Path('C:\\Users\\prisc\\Downloads\\Modelo_spaCy\\model_saved')
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

## Importando o Modelo

In [ ]:
# Load the saved model and predict

print("Loading from", output_dir)
nlp_updated = spacy.load(output_dir)
doc = nlp_updated("Eu gostei da Cachaça Artesanal criada em Minas Gerais" )
print("Entidades", [(ent.text, ent.label_) for ent in doc.ents])